# Import libraries to preprocess data
 - pandas
 - numpy

In [1]:
import pandas as pd
import numpy as np

# Read dataset
the dataset is from kaggle. It is a table of information. 

In [2]:
data = pd.read_table('dataset/training.txt', names=["y", "x"])
y, x = data["y"], data["x"]
# x, y

# Let's prepare dataset

In [3]:
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


# Now I have to create vectors for all sequences


In [4]:
tok = Tokenizer()
tok.fit_on_texts(x)
# tt.texts_to_sequences(t)
seq  = tok.texts_to_sequences(x)
all_x = pad_sequences(seq,maxlen = 5000 )
word_index = tok.word_index
# splitting data in two category positive or negative
all_y = to_categorical(y, num_classes=2)
# all_x, all_y

# generating test, train dataset
since all positive data and negative are together but not shuffled, we gotta shuffle it here. Then we split dataset into test and train parts

# hgdsghdshgd

In [5]:
all_data = np.array(list(zip(all_x, all_y)))
np.random.shuffle(all_data)
r_x, r_y = all_data.T
x_train = r_x[:6000]
x_test  = r_x[6000:]
y_train = r_y[:6000]
y_test  = r_y[6000:]

# x_train, y_train, x_test, y_test

## checking max len of sentence in evaluation dataset

In [6]:
try:
    del data
    del all_data
except:
    pass

# Let's write a model to train these data

In [7]:
from keras.layers import Dense, Activation, Conv1D, MaxPool1D, Embedding, LSTM, SimpleRNN, Flatten, Input

from keras.models import Sequential


In [8]:
xtr_out = np.array([[*i] for i in x_train])
ytr_out = np.array([[*i] for i in y_train])
xts_out = np.array([[*i] for i in x_test])
yts_out = np.array([[*i] for i in y_test])

In [9]:
model = Sequential()

model.add(Embedding(len(word_index) + 1, 1024, input_length=len(x_train[0])))
model.add(Conv1D(2,3))
model.add(MaxPool1D((2)))
model.add(Flatten())
model.add(Dense(2, activation = "sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5000, 1024)        2275328   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4998, 2)           6146      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2499, 2)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4998)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 9998      
Total params: 2,291,472
Trainable params: 2,291,472
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(xtr_out, ytr_out, batch_size= 5, epochs= 10)
model.evaluate(xts_out, yts_out,)
# y_train[0].shape
# x_train[0].shape
# x_train

Epoch 1/10
6000/6000 [==============================] - 791s - loss: 0.1574 - acc: 0.9282   
Epoch 2/10
6000/6000 [==============================] - 1085s - loss: 0.0396 - acc: 0.9883  
Epoch 3/10
5115/6000 [========================>.....] - ETA: 239s - loss: 0.0288 - acc: 0.9918

In [15]:
model.evaluate(xts_out, yts_out,batch_size=918)

918/918 [==============================] - 0s


[0.22197701036930084, 0.89542484283447266]

In [33]:
import time

In [16]:
predict = pd.read_table("./dataset/kaggle.txt", names = ['x'])
predict = predict['x']

In [17]:
tk = Tokenizer()
tk.fit_on_texts(predict)
# tt.texts_to_sequences(t)
seq  = tok.texts_to_sequences(predict)

seq



[[1,
  114,
  181,
  124,
  158,
  162,
  1,
  18,
  89,
  54,
  15,
  116,
  99,
  299,
  115,
  75,
  51,
  12,
  300,
  15,
  1556,
  57,
  19,
  1730,
  2,
  1592,
  3,
  41,
  710,
  22,
  1,
  650,
  117,
  1,
  313,
  391,
  19,
  89,
  19,
  307,
  1,
  26,
  37,
  1,
  324,
  59,
  770,
  328,
  691,
  1,
  317,
  11,
  2,
  299,
  27,
  397,
  1,
  11,
  93,
  46,
  56,
  68,
  38,
  108,
  58,
  34,
  1,
  109,
  304,
  439,
  394,
  88,
  363,
  1,
  83,
  1,
  107,
  14,
  126,
  19,
  38,
  27,
  92,
  3,
  153,
  3,
  167,
  3,
  13,
  32,
  127,
  1155,
  3,
  1,
  14,
  32,
  1219,
  1,
  115,
  18,
  104,
  318,
  113,
  1,
  308,
  13,
  844,
  31,
  434,
  37,
  21,
  2155,
  75,
  3,
  1,
  129,
  13,
  1,
  11,
  1,
  115,
  18,
  104,
  318,
  12,
  91,
  1,
  59,
  12,
  27,
  30,
  105,
  91,
  329,
  65,
  106,
  2,
  110,
  12,
  508,
  12,
  91,
  3,
  58,
  15,
  1,
  11,
  1,
  11,
  104,
  318,
  56,
  55,
  843,
  45,
  1868,
  18,
  51,
  12,
  323,
  5

In [19]:
all_x = pad_sequences(seq,maxlen = 5000 )

In [21]:
res = model.predict(x=all_x)
# max([len(i) for i in predict])

In [22]:
for i, e in enumerate(res[:10]):
    n, p = e
    if n > p:
        print("Negative : ", end= " ")
    else:
        print("positive :" , end = " ")
    print(predict[i])
    

Negative :   I don't care what anyone says, I like Hillary Clinton.
have an awesome time at purdue!..
Yep, I'm still in London, which is pretty awesome: P Remind me to post the million and one pictures that I took when I get back to Markham!...
Have to say, I hate Paris Hilton's behavior but I do think she's kinda cute..
i will love the lakers.
I'm so glad I love Paris Hilton, too, or this would be excruciating.
considering most Geico commericals are stupid...
i liked MIT though, esp their little info book(
Before I left Missouri, I thought London was going to be so good and cool and fun and a really great experience and I was really excited.
I still like Tom Cruise.
well, i had a piece of crap toyota celica but it died in portland and i got a ford ranger..
i love angelina jolie.
I still like Tom Cruise.
UCLA is beautiful.
I think Angelina Jolie is so much more beautiful than Jennifer Anniston, who, by the way, is majorly OVERRATED.
Angelina Jolie is beautiful.
and honda's are awesome: